# AtUproot - tutorial

## Concept

Take alphatwirl's `BEvents` and change the concept of `events` to `blocks` of events. The `tree` object becomes an `uproot.tree` with `__getattr__` using `uproot.tree.array` to an event attribute for a block of events. The array is then cached for further use

In [1]:
def _get_branch(self, name):
    if name in self._branch_cache:
        branch = self._branch_cache[name]
    else:
        self.entrystart = self.iBlock * self.blocksize
        self.entrystop = min((self.iBlock+1) * self.blocksize, self.nEvents)
        self.size = self.entrystop - self.entrystart
        branch = self.tree.array(name,
                                 entrystart = self.entrystart,
                                 entrystop = self.entrystop)
        self._branch_cache[name] = branch
    return branch

Iteration over the `BEvents` would be over blocks rather than events. The `_branch_cache` is cleared on each iteration of the loop to clear space in memory and for the new events to be read in

In [2]:
def __iter__(self):
    for self.iBlock in range(self.nBlocks):
        self._branch_cache = {}
        yield self
    self.iBlock = -1

`EventBuilder` and `EventBuilderConfigMaker` are adjusted to accommodate the new `BEvents`

`AtUproot` is the interface to use `alphatwirl` with the new `BEvents`

`Dataset` is updated just for my own personal use

Readers are coded in the same way, but the implementation must accommodate the array / JaggedArray format of branches. My choice of implementation is to pass the content, starts and stops of Jagged arrays to a function for jit with numba. Numba can speedup for loops to the same speed as normal numpy functions. However, they are very flexible and quick to write / understand (which I struggle with sometimes with a chain of numpy commands)

### Certified lumi checker

In [3]:
from numba import njit, boolean
import numpy as np

@njit
def is_certified_lumi(runs, lumis, cert_runs, cert_lumis):
    nev = runs.shape[0]
    is_certified = np.ones(nev, dtype=boolean)

    for iev in range(nev):
        # run not in list, skip
        passed = False
        for irun in range(cert_runs.shape[0]):
            if runs[iev] != cert_runs[irun]:
                continue

            cert_lumi_range = cert_lumis[irun]
            for ibin in range(cert_lumi_range.shape[0]):
                if cert_lumi_range[ibin,0] <= lumis[iev] <= cert_lumi_range[ibin,1]:
                    passed = True
                    break

            if passed:
                break
        is_certified[iev] = passed

    return is_certified

### Selection on Jagged array

In [4]:
import uproot

@njit
def create_new_stops(selection, starts, lens):
    nev = lens.shape[0]
    new_stops = np.zeros(nev, dtype=int32)

    count = 0
    for iev in range(nev):
        for ij in range(lens[iev]):
            rij = starts[iev]+ij
            if selection[rij]:
                count += 1
        new_stops[iev] = count
    return new_stops

def test_new_stops(self, ref_branch):
    new_stops = create_new_stops(
        self.selection, ref_branch.starts, ref_branch.stops-ref_branch.starts,
    )
    new_starts = np.roll(new_stops, 1)
    new_starts[0] = 0
    
    array = uproot.interp.jagged.JaggedArray(
        ref_branch.content[self.selection],
        new_starts,
        new_stops,
    )

## Setup

Clone the repository, into branch devs for now

atuproot comes with a setup script (shamelessly taken from FAST-RA1). I normally have a miniconda environment ready with everything and use that.

Here we can just use SWAN's built in ROOT and pip.

In [5]:
!git clone -b devs https://github.com/shane-breeze/atuproot atuproot

fatal: destination path 'atuproot' already exists and is not an empty directory.


In [6]:
!pip install --user numpy icc-rt numba alphatwirl==0.16.0 pandas pyyaml uproot rootpy

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
import sys
sys.path.append("atuproot/")
sys.path.append("atuproot/atuproot/")
sys.path.append("atuproot/sequence/")

## Run the code

There is a single script to run the atuproot interface to alphatwirl:
`python run.py --blocksize 100000 --mode multiprocessing`

The following will do what's done inside the `run.py` script (without using a command line parser)

In [8]:
from datasets.datasets import get_datasets
datasets = get_datasets(path="atuproot/datasets/datasets.yaml")
for d in datasets:
    print d

Dataset(name = 'MET_Run2016B_v2', parent = 'MET', isdata = True, xsection = None, lumi = 5930, energy = 13000, sumweights = 35987712.0, associates = MET_Run2016B_v2)
Dataset(name = 'ZJetsToNuNu_Pt-250To400', parent = 'ZJetsToNuNu', isdata = False, xsection = 6.219, lumi = 5930, energy = 13000, sumweights = 5163758.924870491, associates = ZJetsToNuNu_Pt-250To400)


In [9]:
from atuproot.AtUproot import AtUproot
process = AtUproot("output",
                  quiet = False,
                  parallel_mode = "multiprocessing",
                  process = 4,
                  max_blocks_per_dataset = -1,
                  max_blocks_per_process = -1,
                  blocksize = 500000,
                  profile = False,
                  profile_out_path = "profile.txt")

Welcome to JupyROOT 6.12/06


In [10]:
from sequence.Readers import ScribblerWrapper
from sequence.sequence import sequence
reader_collector_pairs = [(ScribblerWrapper(rc[0]), rc[1]) for rc in sequence]

In [11]:
for rc in reader_collector_pairs:
    print rc[0].scribbler
    print rc[1]
    print

NullCollector()

NullCollector()

NullCollector()

NullCollector()

NullCollector()

NullCollector()

NullCollector()

NullCollector()

NullCollector()

NullCollector()

NullCollector()

NullCollector()

NullCollector()

NullCollector()

NullCollector()

NullCollector()

NullCollector()




In [ ]:
process.run(datasets, reader_collector_pairs)


Fri Aug 10 00:12:15 2018
        0 /       16 (  0.00%) MET_Run2016B_v2 

Fri Aug 10 00:12:15 2018
        0 /       16 (  0.00%) MET_Run2016B_v2 
        0 /       17 (  0.00%) MET_Run2016B_v2 

Fri Aug 10 00:12:16 2018
        0 /       16 (  0.00%) MET_Run2016B_v2 
        0 /       17 (  0.00%) MET_Run2016B_v2 
        0 /       16 (  0.00%) MET_Run2016B_v2 

Fri Aug 10 00:12:16 2018
        0 /       16 (  0.00%) MET_Run2016B_v2 
        0 /       17 (  0.00%) MET_Run2016B_v2 
        0 /       16 (  0.00%) MET_Run2016B_v2 
        0 /        6 (  0.00%) MET_Run2016B_v2 

Fri Aug 10 00:13:17 2018
        8 /       16 ( 50.00%) MET_Run2016B_v2 
        8 /       17 ( 47.06%) MET_Run2016B_v2 
        3 /       16 ( 18.75%) MET_Run2016B_v2 
        5 /        6 ( 83.33%) MET_Run2016B_v2 

Fri Aug 10 00:13:18 2018
        6 /        6 (100.00%) MET_Run2016B_v2 
        8 /       16 ( 50.00%) MET_Run2016B_v2 
        8 /       17 ( 47.06%) MET_Run2016B_v2 
        3 /       16 ( 18.75

## Event object

In [ ]:
import uproot
from atuproot.BEvents import BEvents
events = BEvents(uproot.open(datasets[0].files[0])["Events"], blocksize=100, maxBlocks=1, start=0)
event = events[0]

In [ ]:
event.Jet_pt 

## CertifiedLumiChecker

In [ ]:
from sequence.sequence import certified_lumi_checker

In [ ]:
# %load atuproot/sequence/Readers/CertifiedLumiChecker.py
import json
import numpy as np
from numba import njit, boolean

class CertifiedLumiChecker(object):
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def begin(self, event):
        self.runs, self.lumi_list = read_json(self.lumi_json_path)

    def event(self, event):
        event.IsCertified = is_certified_lumi(event.run, event.luminosityBlock,
                                              self.runs, self.lumi_list)

@njit
def is_certified_lumi(runs, lumis, cert_runs, cert_lumis):
    nev = runs.shape[0]
    is_certified = np.ones(nev, dtype=boolean)

    for iev in range(nev):
        # run not in list, skip
        passed = False
        for irun in range(cert_runs.shape[0]):
            if runs[iev] != cert_runs[irun]:
                continue

            cert_lumi_range = cert_lumis[irun]
            for ibin in range(cert_lumi_range.shape[0]):
                if cert_lumi_range[ibin,0] <= lumis[iev] <= cert_lumi_range[ibin,1]:
                    passed = True
                    break

            if passed:
                break
        is_certified[iev] = passed

    return is_certified

def read_json(path):
    with open(path, 'r') as f:
        data = json.load(f)
    runs = np.array(sorted(map(int, data.keys())))
    lumis = [np.array(data[str(r)], dtype=int) for r in runs]
    return runs, lumis


In [ ]:
certified_lumi_checker.begin(event)
certified_lumi_checker.event(event)

In [ ]:
print event.IsCertified

## Collections

In [ ]:
from sequence.sequence import collection_creator

In [ ]:
collection_creator.event(event)

In [ ]:
print event.Jet # Collection(name, ref_name, selection)

In [ ]:
print event.Jet.pt

## Skim collections

In [ ]:
from sequence.sequence import skim_collections

In [ ]:
skim_collections.begin(event)
skim_collections.event(event)
skim_collections.end()

In [ ]:
print event.JetSelection # Collection(name, ref_name, selection)

In [ ]:
try:
    print event.JetSelection_pt
except AttributeError:
    print "No attribute JetSelection_pt"
print event.JetSelection.pt
print event.JetSelection_pt

* A selection is applied to each object in a collection resulting in a boolean array
    * Selections are applied as logical operations between the contents of jagged arrays: e.g. `(jet.pt > 40.) & (abs(jet.eta) < 2.4)`
* This array is stored in the new collection along with the reference collection's name. No new arrays are created
* When an attribute of the new collection is called:
    * the boolean array is applied to the reference collection's attribute
    * new starts and stops are generated
    * a jagged array is created and cached for later use

## Cross cleaning

In [ ]:
from sequence.sequence import jet_cross_cleaning

In [ ]:
jet_cross_cleaning.event(event)

In [ ]:
event.JetSelection.pt

* Jets are removed if they overlap with muons, electron or photons with a DeltaR cone of 0.4
* The JetSelection collection's boolean array is updated to reject these jets

## Event selection

In [ ]:
from atuproot.EventBuilderConfigMaker import EventBuilderConfig

config = EventBuilderConfig(
    inputPaths = [datasets[0].files[0]],
    treeName = "Events",
    maxBlocks = 1,
    start = 0,
    blocksize = 10000,
    dataset = datasets[0],
    name = datasets[0].name,
)

In [ ]:
events = BEvents(uproot.open(datasets[0].files[0])["Events"], blocksize=10000, maxBlocks=1, start=0)
event = events[0]
event.config = config

In [ ]:
for rc in reader_collector_pairs:
    rc[0].begin(event)
    rc[0].event(event)
    if hasattr(rc[0], "end"):
        rc[0].end()

In [ ]:
event.Cutflow_Monojet

* `SelectionProducer` takes a list of cuts (takes the logical and of all these selections applied to the event) and creates a boolean array
    * The selections are applied as logical operations between arrays
* The selection is not applied to the event and should be used with reader-collector pairs allowing for multiple cutflows in 1 run
* If the selection reduces the number of events to a small enough value, these events can be exported into dataframes for further manipulation / plotting (e.g. machine learning
* Currently I have a collector which histograms the arrays and then draws the distributions (standard alphatwirl aggregation tools should be fairly simple to adopt with atuproot)

In [ ]:
print event.METnoX.pt
print event.METnoX.pt.shape

In [ ]:
import numpy as np
print event.Cutflow_Monojet
print event.Cutflow_Monojet.shape
print np.argwhere(event.Cutflow_Monojet)

In [ ]:
print event.METnoX.pt[event.Cutflow_Monojet]
print event.METnoX.pt[event.Cutflow_Monojet].shape

In [ ]:
print event.METnoX.pt[1821]

## Limitations

1. Can use up a lot of memory
    1. If branches are only accessed in one reader and not in any others then theres a function in `BEvents` allowing the deletion of branches from the cache
    1. `blocksize` can be reduced to load fewer event in per block. This can reduce the performance if you ask for too few
    1. Some samples use more memory than others. Request more memory for those samples if possible
    1. Profile the memory usage with `memory_profiler`
1. No support for chaining files (yet)
    1. So far there's not support for chaining files. The number of files per process should always be 1
    1. I've hadded my root files (into several larger files). ~300-400 files altogher = 300-400 jobs
1. Speed
    1. Profiling the code shows that the major bottleneck is reading in the trees with uproot (even if it is really fast)
    1. lzma compression slows this reading process down. If possible lower the compression of your files saving uproot to do it on each run
    1. Good side of this is that alphatwirl's readers / collectors don't impact the speed significantly. Especially with the use of numba's jit compliation
1. Reading large root files
    1. `uproot` creates a memory mapping of root files. Therefore files larges than ~4 GB can't be mapped on 32-bit systems (if anyone is usings these still?)
    1. `ulimit -v` can also limit this. Especially annoying if the cluster has a hard limit on the maximum vitrual memory